In [164]:
import pandas as pd
import numpy as np 
from datetime import datetime

In [170]:
data = pd.read_csv("_2019-7-16_1826.csv")

In [181]:
#Узнаю id всех мэнеджеров 
pd.unique(data["manager_id"]).tolist()
managers_list = pd.unique(data["manager_id"]).tolist()


In [179]:
#Посмотрел на причины отмены и их количество у менеджера, который фродит, самый популярный "already-buyed", что в целом логично
fr = pd.unique(data.loc[data['manager_id'] == 20429].loc[data['state'] == "canceled"]["extended"]).tolist()
for x in fr:
    s = data.loc[data['manager_id'] == 20429].loc[data["extended"] == x]
    print(len(s),x)
    
    

40 error-test
50 prices-did-not-suit
89 already-buyed
68 no-product
18 no-call
13 delivery-did-not-suit
69 other-reason
4 double
6 cancel-invoice
10 not-needed-yet
5 cant-deliver
2 client-refused
3 dramatical-delay
4 invalid-price-on-website
2 refused-payment-type
1 not-selfpickup


In [214]:
#Мэнеджер у которого нет отмененых заказов - хороший
#Оставим только тех, у кого есть ненулевое количество
#Посчитаем отношение общего колв-ва отменненых к общему для каждого менеджера
vals = []


for x in managers_list:
    cancel = len(data.loc[data['manager_id'] == x].loc[data['state'] == "canceled"])
    totall = len(data.loc[data['manager_id'] == x])
    cout_buyed = len(data.loc[data['manager_id'] == x].loc[data['extended'] == "already-buyed"])
    if cancel != 0:
        ratio1 = cancel/totall
        ratio2 = cout_buyed/cancel
    val =  {"manager":x, "total":totall, "cancel":cancel, "bad_cancel":cout_buyed, "ratio1":ratio1, "ratio2":ratio2}
    
    vals.append(val)
df2 = pd.DataFrame(vals) 


In [223]:
#Оставит только тех у кого есть отмена заказа, из-за того, что он уже куплен 

df3 = df2.loc[df2["bad_cancel"] !=0]
df3

,bad_cancel,cancel,manager,ratio1,ratio2,total
0,392,4251,1,0.769830,0.092214,5522
1,6,44,25260,0.709677,0.136364,62
2,14,150,25259,0.447761,0.093333,335
3,17,230,17075,0.363924,0.073913,632
4,2,13,26545,0.464286,0.153846,28
5,5,70,25257,0.167464,0.071429,418
6,33,388,4932,0.349550,0.085052,1110
7,1,4,26267,1.000000,0.250000,4
9,11,109,5188,0.459916,0.100917,237
10,26,224,10787,0.312413,0.116071,717


In [228]:
(df3.loc[df3["bad_cancel"] !=0]).loc[df3["manager"]==20429]

,bad_cancel,cancel,manager,ratio1,ratio2,total
12,89,384,20429,0.370299,0.231771,1037


In [235]:
#ratio1 показывает на сколько в целом эффективен менеджер, чем отношение меньше тем лучше
#ratio2 на сколько он подозрителен
#Предлагаю обратить  внимание на тех менеджеров, у которых ratio2 выше чем coef
#В зависимости от размеров фрэйма можно попытаться определить значимое количество подозрительных продаж

coef = (df3["ratio2"].median() +df3["ratio2"].mean())/2
df3.loc[df3["ratio2"] >= coef]

,bad_cancel,cancel,manager,ratio1,ratio2,total
7,1,4,26267,1.000000,0.250000,4
12,89,384,20429,0.370299,0.231771,1037
14,1,2,15579,0.181818,0.500000,11
24,21,67,31310,0.262745,0.313433,255
26,2,5,33362,0.333333,0.400000,15
31,1,1,37754,0.076923,1.000000,13
33,24,69,26574,0.305310,0.347826,226
39,21,121,38948,0.372308,0.173554,325
40,25,98,40333,0.497462,0.255102,197
50,10,44,41428,0.346457,0.227273,127
